In [ ]:
import cv2
import numpy as np
from djitellopy import Tello

In [ ]:
# 배터리 체크
def battery_check() : 
  drone = Tello()
  drone.connect()

  power = drone.get_battery()
  if power < 30 : print("배터리 부족", power)
  else : print("배터리", power)
  drone.end()
  return power
battery_check()

In [ ]:
#  -- 초기값 설정 --
# 스타트flag
start_counter = 1
# 프레임 flag - 이때 딥러닝 예측도 실행해야됨
frame_flag = 100
# 임계값 조절
tolerance_x = 5
tolerance_y = 5
# 속도 한계값
slowdown_threshold_x = 15
slowdown_threshold_y = 15
# 속도
drone_speen_x = 10
drone_speen_y = 10
# 화면 조절
set_point_x = 960/2
set_point_y = 720/2
hull = 0
# x,y,w,h 전역변수
# global x, y, w, h
x,y,w,h = 0,0,0,0


# 드론 초기화
drone = Tello()
drone.connect()

# 스트림 연결 오류 
drone.streamon()

# 모델 임포트
# faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# -- 영상 루프 시작 --
while True :
  # takeoff 설정
  if start_counter == 0 :
    drone.takeoff()
    print('takeoff')
    start_counter = 1

  # 프레임 읽어오기
  frame = drone.get_frame_read().frame
  cv2.circle(frame, (int(set_point_x), int(set_point_y)), 12, (255,0,0), 1) # 화면 중간에 원표시
  
  # flalg 설정
  frame_flag -=1
  # print("frame_flag", frame_flag)
  if frame_flag == 0 :
    print("frame_flag 0일때 ")



    # -- 필터처리 --
    img_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    img_hsv = cv2.fastNlMeansDenoisingColored(img_hsv,None,10,10,7,21)
    lower = np.array([0,48,80], dtype="uint8")
    upper = np.array([20,255,255], dtype="uint8")
    img_hand = cv2.inRange(img_hsv,lower,upper)
    
    #경계선 찾음
    contours, _ = cv2.findContours(img_hand, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    # 가장 큰 영역 찾기
    max = 0
    maxcnt = None
    
    # 이건 뭔지 모름
    for cnt in contours :
      area = cv2.contourArea(cnt)
      if(max < area) :
        max = area
        maxcnt = cnt
    
    # 이것도 뭔지 모름 암튼 예외처리
    if maxcnt != np.array([]):
      mask = np.zeros(frame.shape).astype(frame.dtype)
      # 경계선 내부 255로 채우기
      color = [255, 255, 255]
      cv2.fillPoly(mask, [maxcnt], color)
      img_hand = cv2.bitwise_and(frame, mask)

    hull = cv2.convexHull(maxcnt)
    x, y, w, h = cv2.boundingRect(hull)
    print(x,y,w,h)
    frame_flag = 100

  # 도출된 값으로 사각형 그리기
  cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0),3 )
  cv2.circle(frame, (int(x+w/2) , int(y+h/2)), 12, (255,0,0), 1) # 얼굴 중심 표시

  # -- 드론 제어 --
  #  얼굴중심과 화면중심의 차를 계산
  distance_x = x+w/2 - set_point_x
  distance_y = y+h/2 - set_point_y

  up_down_velocity = 0
  right_left_veiocity = 0
  for_back_veiocity = 0

# 드론 좌우 이동
  if distance_x < -tolerance_x:
    print("left move")
    right_left_veiocity = - drone_speen_x
  elif distance_x > tolerance_x:
    print("right move")
    right_left_veiocity = drone_speen_x
  else :
    print("OK")

  # 드론 상하 이동
  if distance_y < -tolerance_y:
    print("up move")
    up_down_velocity = drone_speen_y
  elif distance_y > tolerance_y:
    print("down move")
    up_down_velocity = - drone_speen_y
  else :
    print("OK")

  # 드론 앞뒤 이동
  if w*h < 960*720/2:
    for_back_veiocity = 10
  elif w*h > 960*720/2:
    for_back_veiocity = -10
  elif w*h <= 960*720 :
    for_back_veiocity = 10
  else:
    print("OK")



#  임계치 이상 벗어나면 속도 조정 
  if abs(distance_x) < slowdown_threshold_x:
    right_left_veiocity = int(right_left_veiocity / 2)
  if abs(distance_y) < slowdown_threshold_y:
    up_down_velocity = int(up_down_velocity / 2)

  #드론 움직이기
  drone.send_rc_control(right_left_veiocity, 0, up_down_velocity, 0)

  # 비디오 띄우기
  cv2.imshow("Video", frame)
  # 키 설정
  key = cv2.waitKey(1)
  if key == ord('q'):
    break

cv2.destroyAllWindows()
drone.streamoff()
drone.end()

In [ ]:
def process_data(path):
    x = []
    y = []
    dir_list = os.listdir(path)
    #print(os.listdir(path))
    for i in range(0,len(dir_list)):
        # if i == 1:
        #     break
        dir_path = path + "/" + dir_list[i]
        #print(os.listdir(dir_path))
        dir_name = os.listdir(dir_path)
        
        for j in range(len(dir_name)):
            
            full_dir_path = dir_path + "/" + dir_name[j]
            print(full_dir_path)
            img = cv2.imread(full_dir_path,cv2.IMREAD_COLOR)
            
            # 색 변경
            img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

            img_hsv = cv2.fastNlMeansDenoisingColored(img_hsv,None,10,10,7,21)
            lower = np.array([0,48,80], dtype="uint8")
            upper = np.array([20,255,255], dtype="uint8")
            img_hand = cv2.inRange(img_hsv,lower,upper)
            

            #경계선 찾음
            contours, hierarchy = cv2.findContours(img_hand, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            # 가장 큰 영역 찾기
            max = 0
            maxcnt = None
            
            for cnt in contours :
                area = cv2.contourArea(cnt)
                if(max < area) :
                    max = area
                    maxcnt = cnt
            #print("maxcnt : ",maxcnt)
            if maxcnt != np.array([]):
                mask = np.zeros(img.shape).astype(img.dtype)
                #print(mask)
                # 경계선 내부 255로 채우기
                color = [255, 255, 255]
                cv2.fillPoly(mask, [maxcnt], color)
                img_hand = cv2.bitwise_and(img, mask)
                
                
                img_hand = cv2.resize(img_hand, (100,100))
                cv2.imwrite('image.png', img_hand)
                x.append(img_hand)
                y.append(i)
            else:
                continue
            
    #print(x.shape)
    #print(y.shape)
    return np.array(x),np.array(y)

#process_data("./data/train")


In [1]:
import cv2
import numpy as np
from djitellopy import Tello

from tensorflow.keras import models
#  -- 초기값 설정 --
# 스타트flag
start_counter = 1
# 프레임 flag - 이때 딥러닝 예측도 실행해야됨
frame_flag = 70
# 임계값 조절
tolerance_x = 5
tolerance_y = 5
# 속도 한계값
slowdown_threshold_x = 13
slowdown_threshold_y = 13
# 속도
drone_speen_x = 13
drone_speen_y = 13
# 화면 조절
set_point_x = 960/2
set_point_y = 720/2
hull = 0
# x,y,w,h 전역변수
# global x, y, w, h
x =0
y =0
w =0
h =0
accuracy_num = ''
img_hand = 0

# 드론 초기화
drone = Tello()
# drone.connect()

# # 스트림 연결 오류 
# drone.streamon()

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,1200)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,1600)

# 모델 임포트
# faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
model = models.load_model("last_model.h5")
#x,y,w,h = 0
# ------------ 영상 루프 시작 ------------
while True :
  # 프레임 읽어오기
  # frame = drone.get_frame_read().frame
  # takeoff 설정
  if start_counter == 0 :
    drone.takeoff()
    # drone.move_up(80)
    print('takeoff')
    start_counter = 1

  _, frame = cap.read()

  cv2.circle(frame, (int(set_point_x), int(set_point_y)), 12, (255,0,0), 1) # 화면 중간에 원표시
  cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 3 )
  # cv2.putText(frame,f'accuracy = ',accuracy_num,(x+5,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 0.5, cv2.LINE_AA)
  # flalg 설정
  frame_flag -=1
  if frame_flag < 0:
    frame_flag =70
  #print(frame_flag)
  # print("frame_flag", frame_flag)
  if frame_flag == 0 :
    # ------------ 데이터 가공 ------------
    img_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    img_hsv = cv2.fastNlMeansDenoisingColored(img_hsv,None,10,10,7,21)
    lower = np.array([0,48,80], dtype="uint8")
    upper = np.array([20,255,255], dtype="uint8")
    img_hand = cv2.inRange(img_hsv,lower,upper)
    
    #경계선 찾음
    contours, _ = cv2.findContours(img_hand, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    # 가장 큰 영역 찾기
    max = 0
    maxcnt = None
    
    # 채워주기
    for cnt in contours :
      area = cv2.contourArea(cnt)
      if(max < area) :
        max = area
        maxcnt = cnt
    
    # 예외처리
    if maxcnt != np.array([]):
      mask = np.zeros(frame.shape).astype(frame.dtype)
      # 경계선 내부 255로 채우기
      color = [255, 255, 255]
      cv2.fillPoly(mask, [maxcnt], color)
      img_hand = cv2.bitwise_and(frame, mask)

      #print("전",img_hand.shape)
      img_hand_r = cv2.resize(img_hand, (100,100))
      #print(img_hand.shape)
    else :
      continue
    # ------------ 예측 실행 ------------
    # img_hand = np.array([img_hand])
    pred = model.predict(np.array([img_hand_r]))
    print("예측 값 ==>  ",pred)
    # accuracy_num = str(pred[0][1])
    print(accuracy_num)
    pred = np.argmax(pred)
    print("1 : 얼굴 인식,  0 : 인식 실패 ==> ",pred)
    
    if pred == 1 :
      hull = cv2.convexHull(maxcnt)
      x, y, w, h = cv2.boundingRect(hull)
      print("x,y,w,h : ", x,y,w,h)
      cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 3 )
      cv2.circle(frame, (int(x+w/2) , int(y+h/2)), 12, (0,255,0), 1) # 얼굴 중심 표시
      
      frame_flag = 100

      # 도출된 값으로 사각형 그리기
      

      # ------------ 드론 제어 ------------
      #  얼굴중심과 화면중심의 차를 계산
      distance_x = x+w/2 - set_point_x
      distance_y = y+h/2 - set_point_y

      up_down_velocity = 0
      right_left_veiocity = 0
      for_back_veiocity = 0

    # 드론 좌우 이동
      if distance_x < -tolerance_x:
        print("left move")
        right_left_veiocity = - drone_speen_x
      elif distance_x > tolerance_x:
        print("right move")
        right_left_veiocity = drone_speen_x
      else :
        print("OK")

      # 드론 상하 이동
      if distance_y < -tolerance_y:
        print("up move")
        up_down_velocity = drone_speen_y
      elif distance_y > tolerance_y:
        print("down move")
        up_down_velocity = - drone_speen_y
      else :
        print("OK")

      # 드론 앞뒤 이동 및 프레임 크면 넘기기
      # if w*h < 960*720/2:
      #   for_back_veiocity = 10
      # elif w*h > 960*720/2:
      #   for_back_veiocity = -10
      # elif w*h > 960*720 :
      #   for_back_veiocity = 10
      # elif w*h >= 950*700 :
      #   continue
      # else:
      #   print("OK")

      #  임계치 이상 벗어나면 속도 조정 
      if abs(distance_x) < slowdown_threshold_x:
        right_left_veiocity = int(right_left_veiocity / 2)
      if abs(distance_y) < slowdown_threshold_y:
        up_down_velocity = int(up_down_velocity / 2)

      #드론 움직이기
      # drone.send_rc_control(right_left_veiocity, 0, up_down_velocity, 0)
    else :
      # drone.send_rc_control(0, 0, 0, 0)
      print("0일때 물체 추적")
      continue


  # 비디오 띄우기
  cv2.imshow("Video", frame)
  # cv2.imshow("Video", img_hand)
  # 키 설정
  key = cv2.waitKey(1)
  if key == ord('q'):
    break


drone.streamoff()
cv2.destroyAllWindows()
drone.end()

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
C:\Users\nono\AppData\Local\Temp\ipykernel_25324\3198625119.py:92: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if maxcnt != np.array([]):


예측 값 ==>   [[7.349868e-30 1.000000e+00]]

1 : 얼굴 인식,  0 : 인식 실패 ==>  1
x,y,w,h :  447 0 673 720
right move
OK
예측 값 ==>   [[0. 1.]]

1 : 얼굴 인식,  0 : 인식 실패 ==>  1
x,y,w,h :  484 271 215 305
right move
down move
예측 값 ==>   [[1.000000e+00 8.775571e-12]]

1 : 얼굴 인식,  0 : 인식 실패 ==>  0
0일때 물체 추적
예측 값 ==>   [[1.6753401e-07 9.9999988e-01]]

1 : 얼굴 인식,  0 : 인식 실패 ==>  1
x,y,w,h :  336 249 146 304
left move
down move
예측 값 ==>   [[0.00222621 0.9977738 ]]

1 : 얼굴 인식,  0 : 인식 실패 ==>  1
x,y,w,h :  339 252 141 237
left move
down move
예측 값 ==>   [[0. 1.]]

1 : 얼굴 인식,  0 : 인식 실패 ==>  1
x,y,w,h :  661 215 193 290
right move
OK
예측 값 ==>   [[6.1080865e-20 1.0000000e+00]]

1 : 얼굴 인식,  0 : 인식 실패 ==>  1
x,y,w,h :  262 251 223 301
left move
down move
예측 값 ==>   [[1.000000e+00 1.731352e-11]]

1 : 얼굴 인식,  0 : 인식 실패 ==>  0
0일때 물체 추적
예측 값 ==>   [[5.746812e-31 1.000000e+00]]

1 : 얼굴 인식,  0 : 인식 실패 ==>  1
x,y,w,h :  650 283 362 298
right move
down move
예측 값 ==>   [[1.0000000e+00 4.3956871e-10]]

1 : 얼굴 인식,  0 : 인식 실

[INFO] tello.py - 437 - Send command: 'streamoff'


예측 값 ==>   [[9.999784e-01 2.158064e-05]]

1 : 얼굴 인식,  0 : 인식 실패 ==>  0
0일때 물체 추적


OSError: [WinError 10051] 연결할 수 없는 네트워크에서 소켓 작업을 시도했습니다

: 